<a href="https://colab.research.google.com/github/ua-datalab/Geospatial_Workshops/blob/main/notebooks/STAC_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SpatioTemporal Asset Catalog (STAC)
This notebook demonstrates the use of pystac_client python library to crawl through and access geospatial assets from a STAC complient API.

In [ ]:
# Install pystac_client. This library is used to crawl SpatioTemporal Asset Catalogs (STAC)
!pip install pystac_client --quiet
!pip install geopandas --quiet
!pip install folium --quiet

In [10]:
#Import the libraries into the current session

import pystac_client
import geopandas
import folium

In [3]:
catalog = pystac_client.Client.open(
    "https://stac.cyverse.org"
)

In [4]:
#Let's look at the collections within the root catalog
collections = list(catalog.get_collections())

# Print the number of collections
print(f"Number of collections in the base catalog: {len(collections)}")

# Print the names (or IDs) and descriptions of each collection
for collection in collections:
    print(f"ID: {collection.id}")

Number of collections in the base catalog: 1
ID: Open Forest Observatory


In [ ]:
#Search the collection to find the number of items

search = catalog.search(collections=["Open Forest Observatory"])
items = search.item_collection()
len(items)

321

In [12]:
# Create a base folium map centered on [0, 0] (change as desired) 
m = folium.Map(location=[0, 0], zoom_start=2)

# Loop over each collection and add bounding boxes
for coll in collections:
    # Each collection could have multiple bboxes, but often there's just one
    for bbox in coll.extent.spatial.bboxes:
        # bbox is typically [west, south, east, north]
        west, south, east, north = bbox

        # Format into a list of lat/lon pairs in Leaflet-friendly order: [lat, lon]
        coords = [
            [south, west],
            [south, east],
            [north, east],
            [north, west],
            [south, west]
        ]

        # Create a polygon for this bbox and add it to the map
        folium.Polygon(locations=coords, fill=False).add_to(m)

# Show the map
m


In [13]:
#Create a custom spatial and temporal filter to find assets of interest

time_range = "2023-01-01/2024-12-31"
bbox = [-123.621, 38.32, -119.67, 40.293] #SW corner longitude/latitude ; NE corner longitude/latitude

In [14]:
#Search the collection to find imagery assets within my time-range and bounding box.

search = catalog.search(collections=["Open Forest Observatory"], bbox=bbox, datetime=time_range)
items = search.item_collection()
len(items)



167

In [ ]:
items = search.get_all_items()  # Or you can iterate page by page
print(f"Found {len(items)} items.")

# Create another folium map
m_items = folium.Map(location=[0, 0], zoom_start=2)

for item in items:
    # item.geometry should be a valid GeoJSON geometry 
    # (polygon, multipolygon, etc.)
    if item.geometry:
        # Directly add GeoJSON geometry
        folium.GeoJson(
            item.geometry,
            tooltip=f"Item ID: {item.id}"
        ).add_to(m_items)

m_items


/home/jgillan/miniconda3/envs/stac_api/lib/python3.12/site-packages/pystac_client/item_search.py:896: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


Found 167 items.


In [22]:
search = catalog.search(collections=["Open Forest Observatory"])  # Correct ID
items = list(search.get_all_items())  # gather all Items

print(f"Downloaded {len(items)} total items. Now filtering locally...")

# Example: Filter by platform or license
filtered_items = []
for item in items:
    platform = item.properties.get("platform", None)
    license_ = item.properties.get("license", None)
    if platform == "Matrice 210 RTK" and license_ == "CC-BY-SA-4.0":
        filtered_items.append(item)

print(f"Remaining after local filter: {len(filtered_items)} items.")
for item in filtered_items:
    print(item.id, item.properties["platform"], item.properties["license"])


Downloaded 321 total items. Now filtering locally...
Remaining after local filter: 0 items.


In [ ]:
# List all the assets for the selected item

import rich.table

table = rich.table.Table("Asset Key", "Description", "Asset Type" )
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title, asset.media_type)

table

In [ ]:
#Convert the 'rendered preview' asset into a dictionary

selected_item.assets["rendered_preview"].to_dict()

In [ ]:
#Display the 'rendered preview' asset of the item

from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

In [ ]:
#Get the API endpoint (url) of the 'blue' band asset.

selected_item.assets["blue"].href

In [ ]:
##Get some info from the asset without downloading it
## Get response code, file type, file size
## We are looking for HTTP status code of 200

import requests

# Send a HEAD request to get the headers of the file
response = requests.head(selected_item.assets["blue"].href)

# Retrieve the status code
status_code = response.status_code

# Initialize variables for file type and size
file_type = None
file_size_mb = None

# Check if the Content-Type header exists
if 'Content-Type' in response.headers:
    file_type = response.headers['Content-Type']

# Check if the Content-Length header exists and convert it to megabytes
if 'Content-Length' in response.headers:
    file_size_bytes = int(response.headers['Content-Length'])
    file_size_mb = file_size_bytes / (1024 * 1024)  # Convert bytes to megabytes

print(f"Status Code: {status_code}")
print(f"File Type: {file_type}")
print(f"File Size: {file_size_mb:.2f} MB")


In [ ]:
##Pull the selected asset (cloud optimized geotiff) into my notebook

#install and import library for display
!pip install rioxarray --quiet

import rioxarray


#Display the selected asset with coarser resolution.
#The asset is a COG so it has overviews embedded

ds = rioxarray.open_rasterio(
    selected_item.assets["blue"].href, overview_level=2
).squeeze()
img = ds.plot(cmap="viridis", add_colorbar=False)
img.axes.set_axis_off();